In [ ]:
!pip install bertopic
!pip install cudf-cu11 dask-cudf-cu11 --extra-index-url=https://pypi.nvidia.com
!pip install cuml-cu11 --extra-index-url=https://pypi.nvidia.com
!pip install cugraph-cu11 --extra-index-url=https://pypi.nvidia.com
!pip install --upgrade cupy-cuda11x -f https://pip.cupy.dev/aarch64
!pip install umap-learn

In [ ]:
import pandas as pd
import glob
import os
from google.colab import files
from bertopic import BERTopic
import openai
from bertopic.backend import OpenAIBackend
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from umap import UMAP
import openai
import numpy as np
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Upload the file manually
uploaded = files.upload()

# List the uploaded files
df = pd.read_csv("Preprocessed_jobs.csv")

#Isolate input
docs = df['Cleaned Job Description'].tolist()

Saving Preprocessed_jobs.csv to Preprocessed_jobs.csv


In [ ]:
# Setting up a client for ChatGPT with API
client = openai.OpenAI(api_key="xx")
# Fine-tune topic embeddings with GPT
embedding_model = OpenAIBackend(client, "text-embedding-ada-002")
# Fine-tune topic representations with GPT
representation_model = OpenAI(client, model="gpt-3.5-turbo", chat=True)

topic_model_llm = BERTopic(representation_model=representation_model, embedding_model=embedding_model)
topics, probs = topic_model_llm.fit_transform(docs)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("jjzha/jobbert_skill_extraction")
model = AutoModelForMaskedLM.from_pretrained("jjzha/jobbert_skill_extraction")

tokenizer_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/942 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

Some weights of BertForMaskedLM were not initialized from the model checkpoint at jjzha/jobbert_skill_extraction and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
topic_model_SkillSpan = BERTopic(vectorizer_model=vectorizer, embedding_model=model) #Use SkillSpan for word embeddings model
topics, probs = topic_model_SkillSpan.fit_transform(docs)

In [ ]:
topic_model_SkillSpan.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7,-1_paypal_merchants_consumers_credit,"[paypal, merchants, consumers, credit, payment...",[the company paypal has been revolutionizing c...
1,0,454,0_generative_generative ai_model_time,"[generative, generative ai, model, time, aws, ...",[8 years of specific technology domain areas e...
2,1,20,1_bytedance_doubao_together_model,"[bytedance, doubao, together, model, china, an...","[responsibilities established in 2023, the byt..."
3,2,17,2_together_fair_angeles county_los angeles county,"[together, fair, angeles county, los angeles c...",[responsibilities why join us creation is the ...


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("token-classification", model="jjzha/jobbert_skill_extraction")

Device set to use cpu


In [ ]:
from nltk.corpus import stopwords

stop_words = stopwords.words("english")  # Load extensive stopword list

# CountVectorizer to remove stopwords and include bigrams
vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1, 3))

topic_model_SBERT = BERTopic(vectorizer_model=vectorizer, embedding_model="all-MiniLM-L6-v2") #Use SBert for word embeddings
topics, probs = topic_model_SBERT.fit_transform(docs)

In [ ]:
# CountVectorizer to remove stopwords and include bigrams
vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1, 3))

topic_model_SBERT = BERTopic(vectorizer_model=vectorizer, embedding_model="all-mpnet-base-v2") #Larger model for better topic seperation
topics, probs = topic_model_SBERT.fit_transform(docs)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
topic_model_SBERT.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,464,0_experience_data_ai_work,"[experience, data, ai, work, learning, models,...","[5 years of developing, negotiating and execut..."
1,1,19,1_bytedance_including_team_us,"[bytedance, including, team, us, information, ...","[responsibilities established in 2023, the byt..."
2,2,15,2_tiktok_information_us_team,"[tiktok, information, us, team, together, comp...",[responsibilities why join us creation is the ...


In [ ]:
topic_model_SBERT.get_topic(1)

[('bytedance', 0.027337015238096663),
 ('including', 0.02318046958200869),
 ('team', 0.023059923634402766),
 ('us', 0.022712980902263608),
 ('information', 0.019596981328558118),
 ('doubao', 0.01753562725201605),
 ('candidates', 0.016592395809327573),
 ('together', 0.016533977152541672),
 ('model', 0.015254432849627417),
 ('company', 0.015155615627644699)]

In [ ]:
stop_words = stopwords.words("english")  # Load extensive stopword list
# CountVectorizer to remove stopwords and include bigrams
vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1, 3))

topic_model_SBERT2 = BERTopic(vectorizer_model=vectorizer, embedding_model="all-MiniLM-L6-v2", min_topic_size=5, nr_topics=100) #addition of min_topic_size
topics, probs = topic_model_SBERT2.fit_transform(docs)

In [ ]:
topic_model_SBERT2.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,454,0_experience_data_ai_work,"[experience, data, ai, work, learning, models,...","[5 years of developing, negotiating and execut..."
1,1,17,1_tiktok_information_us_together,"[tiktok, information, us, together, team, comp...",[responsibilities why join us creation is the ...
2,2,10,2_bytedance_including_team_us,"[bytedance, including, team, us, information, ...","[responsibilities founded in 2012, bytedances ..."
3,3,10,3_bytedance_team_doubao_us,"[bytedance, team, doubao, us, including, multi...","[responsibilities established in 2023, the byt..."
4,4,7,4_paypal_merchants_consumers_credit,"[paypal, merchants, consumers, credit, payment...",[the company paypal has been revolutionizing c...


In [ ]:
# Standard English Stopwords as a List
custom_stopwords = list(stopwords.words('english'))

# Updated Stopwords List (Generic Words + Extracted Words)
job_posting_stopwords = [
    "experience", "role", "position", "opportunity", "company", "team", "industry",
    "organization", "professional", "required", "desired", "responsibilities", "expectations",
    "preferred", "qualifications", "background", "collaborate", "support", "provide", "including",
    "success", "contribute", "responsible", "plus", "must", "advantage", "growth", "ensure", "join",
    "apply", "drive", "mission", "customer", "benefits", "salary", "pay", "perks", "product",
    "service", "competitive", "leadership", "communication",
    "collaboration", "interpersonal", "team player", "management", "self-motivated",
    "proactive", "independent", "fast-paced", "organizational", "willingness to", "eager to",
    "excellent", "stakeholders"
]

additional_stopwords = ['tiktok', 'instagram', 'facebook', 'sales', 'customers', 'marketing', 'advertising', 'client']

# Merge Standard and Custom Stopwords as a List
stop_words = list(set(custom_stopwords + job_posting_stopwords + additional_stopwords))  # Remove duplicates

# Update CountVectorizer with stop_words as a list
vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1, 4), max_df=0.85)

In [ ]:
#model with updated stopwords vocabulary
topic_model_SBERT3 = BERTopic(vectorizer_model=vectorizer, embedding_model="all-MiniLM-L6-v2", min_topic_size=3) #addition of min_topic_size
topics, probs = topic_model_SBERT3.fit_transform(docs)

In [ ]:
topic_model_SBERT3.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,138,-1_machine learning_generative_generative ai_p...,"[machine learning, generative, generative ai, ...",[position description harvard business school ...
1,0,19,0_financial_investment_global_clients,"[financial, investment, global, clients, netfl...",[smbc group is a toptier global financial grou...
2,1,17,1_together_candidates_los angeles county_angel...,"[together, candidates, los angeles county, ang...",[responsibilities why join us creation is the ...
3,2,15,2_statistical_data science_python_data scientist,"[statistical, data science, python, data scien...",[minimum education bachelors degree or equival...
4,3,14,3_generative_generative ai_aws_cloud,"[generative, generative ai, aws, cloud, engine...",[bachelors degree or equivalent practical expe...
5,4,13,4_health_patients_clinical_healthcare,"[health, patients, clinical, healthcare, disco...",[bring your heart to cvs health. every one of ...
6,5,12,5_generative_generative ai_radian_ai models,"[generative, generative ai, radian, ai models,...","[job summary as a generative ai architect, you..."
7,6,12,6_invisible_employees_cisco_year,"[invisible, employees, cisco, year, clients, s...",[about invisible invisible is the operations i...
8,7,11,7_apple_machine learning_foundation_experiences,"[apple, machine learning, foundation, experien...","[summary posted aug 9, 2024 role number2005618..."
9,8,11,8_waymo_rivian_machine learning_motion,"[waymo, rivian, machine learning, motion, perc...",[what to expect as a member of the autopilot t...


In [ ]:
topic_model_SBERT3.get_topic(7)

[('tiktok', np.float64(0.01692994107519993)),
 ('fair', np.float64(0.007858218780133245)),
 ('los angeles county', np.float64(0.007837037153645731)),
 ('angeles county', np.float64(0.007837037153645731)),
 ('fair chance', np.float64(0.007697860108072057)),
 ('angeles', np.float64(0.007697860108072057)),
 ('los angeles', np.float64(0.007697860108072057)),
 ('los', np.float64(0.007638925643257721)),
 ('county', np.float64(0.007593513009998965)),
 ('chance', np.float64(0.007435323207927331))]

In [ ]:
#BERTopic with seed_words
seed_words = [
    ["LLM", "Transformers", "GPT", "BERT", "T5", "Mixtral"],
    ["Fine-tuning", "LoRA", "Quantization", "RLHF", "Hyperparameter Tuning"],
    ["Vector Databases", "FAISS", "Pinecone", "Semantic Search"],
    ["Prompt Engineering", "Few-shot Learning", "Chain-of-thought"],
    ["Deep Reinforcement Learning", "AutoML", "Active Learning"],
    ["MLOps", "Model Deployment", "TensorRT", "ONNX"],
    ["AI Ethics", "Bias Mitigation", "Explainability", "Fairness"],
]

# Initialize BERTopic with the seed words
topic_model = BERTopic(seed_topic_list=seed_words)

#model with updated stopwords vocabulary
topic_model_SEED = BERTopic(vectorizer_model=vectorizer, embedding_model="all-MiniLM-L6-v2", min_topic_size=3, seed_topic_list=seed_words) #addition of min_topic_size
topics, probs = topic_model_SEED.fit_transform(docs)

# Review discovered topics
topic_model_SEED.get_topics()

{-1: [('data science', np.float64(0.002427299586749155)),
  ('analytics', np.float64(0.0023170530227178444)),
  ('clients', np.float64(0.002206859509773471)),
  ('employees', np.float64(0.0021111099400647477)),
  ('cloud', np.float64(0.002017802947714707)),
  ('security', np.float64(0.002002249300956361)),
  ('operations', np.float64(0.001999383947279674)),
  ('project', np.float64(0.0019853791666264915)),
  ('people', np.float64(0.0019808110409199144)),
  ('youll', np.float64(0.001972194035443971))],
 0: [('staff', np.float64(0.00684276588544105)),
  ('student', np.float64(0.006715631303484342)),
  ('library', np.float64(0.004778794636972256)),
  ('ymca', np.float64(0.004739044435368259)),
  ('director', np.float64(0.004617176903531822)),
  ('students', np.float64(0.004371157941124524)),
  ('procedures', np.float64(0.004171624477080272)),
  ('program', np.float64(0.004009741677674629)),
  ('programs', np.float64(0.0039295941952441835)),
  ('community', np.float64(0.003802708438419866)

In [ ]:
topic_model_SEED.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,143,-1_data science_analytics_clients_employees,"[data science, analytics, clients, employees, ...",[3 years of experience in end to end technical...
1,0,23,0_staff_student_library_ymca,"[staff, student, library, ymca, director, stud...",[come join the kansas humane society pack! we ...
2,1,21,1_data science_analytics_data engineering_sql,"[data science, analytics, data engineering, sq...",[position purpose renown health is looking for...
3,2,17,2_together_candidates_los angeles county_angel...,"[together, candidates, los angeles county, ang...",[responsibilities why join us creation is the ...
4,3,17,3_sap_parts_transformer_software development,"[sap, parts, transformer, software development...","[we help the world run better at sap, we enabl..."
5,4,14,4_scientific_health_healthcare_biology,"[scientific, health, healthcare, biology, comp...",[description are you passionate about transfor...
6,5,13,5_invisible_equipment_employees_cisco,"[invisible, equipment, employees, cisco, elect...",[about invisible invisible is the operations i...
7,6,12,6_aws_genai_ml_cloud,"[aws, genai, ml, cloud, gtm, amazon, never, am...","[7 years of relevant gtm, sales, or consulting..."
8,7,12,7_radian_employee_ai models_artificial,"[radian, employee, ai models, artificial, ai s...",[what to expect this software engineer will be...
9,8,11,8_health_care_patients_pharmacy,"[health, care, patients, pharmacy, patient, pr...",[we are here for lifes journey. where is your ...


In [ ]:
# Define integrated seed words covering both classic & modern AI competencies
seed_words = [
    # Generative AI & LLM Skills
    ["LLM", "Transformers", "GPT", "BERT", "T5", "Mixtral"],
    ["Fine-tuning", "LoRA", "Quantization", "RLHF", "Hyperparameter Tuning"],
    ["Vector Databases", "FAISS", "Pinecone", "Semantic Search"],
    ["Prompt Engineering", "Few-shot Learning", "Chain-of-thought"],

    # Classic Data Science & Machine Learning Skills
    ["Linear Regression", "Logistic Regression", "Random Forest", "SVM"],
    ["Gradient Boosting", "XGBoost", "CatBoost", "LightGBM"],
    ["K-Means", "DBSCAN", "Hierarchical Clustering"],
    ["Feature Engineering", "Dimensionality Reduction", "PCA", "t-SNE"],

    # Programming & Software Engineering
    ["Python", "SQL", "C++", "Java", "R", "Scala"],
    ["TensorFlow", "PyTorch", "JAX", "FastAPI", "LangChain"],

    # MLOps & Deployment
    ["MLOps", "Model Deployment", "Docker", "Kubernetes", "TensorRT", "ONNX"],
    ["Airflow", "MLflow", "Triton Inference Server"],

    # Statistics & Mathematics
    ["Probability", "Linear Algebra", "Optimization", "Bayesian Inference"],
    ["Hypothesis Testing", "A/B Testing", "Time Series Analysis"],

    # Ethics & Governance
    ["AI Ethics", "Bias Mitigation", "Explainability", "Fairness"],
]

# Initialize BERTopic with the seed words
topic_model = BERTopic(seed_topic_list=seed_words)

#model with updated stopwords vocabulary
topic_model_SEED2 = BERTopic(vectorizer_model=vectorizer, embedding_model="all-MiniLM-L6-v2", min_topic_size=3, seed_topic_list=seed_words) #addition of min_topic_size
topics, probs = topic_model_SEED2.fit_transform(docs)

In [ ]:
# Review discovered topics
topic_model_SEED2.get_topic_info()

In [ ]:
import openai  # Zorg dat de OpenAI module is geïmporteerd
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [ ]:
# Setting up a client for ChatGPT with API
client = openai.OpenAI(api_key="xx")
# Fine-tune topic embeddings with GPT
embedding_model = OpenAIBackend(client, "all-MiniLM-L6-v2")
# Fine-tune topic representations with GPT
#representation_model = OpenAI(client, model="gpt-3.5-turbo", chat=True)

stop_words = stopwords.words("english")  # Load extensive stopword list

# CountVectorizer to remove stopwords and include bigrams
vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1, 3))

topic_model_LLMO = BERTopic(vectorizer_model=vectorizer,
                            embedding_model=embedding_model,
                            min_topic_size=5)

topics, probs = topic_model_LLMO.fit_transform(docs)

In [ ]:
topic_model_LLMO.get_topic(1)

[('ai', 0.03042456209902407),
 ('generative', 0.01587411542974128),
 ('generative ai', 0.014507553336820201),
 ('solutions', 0.0133150014526315),
 ('experience', 0.009830584072765507),
 ('business', 0.009281293834906959),
 ('ai solutions', 0.00812398357680608),
 ('design', 0.007986155318406152),
 ('models', 0.007790284076752635),
 ('data', 0.006918937972592953)]

In [ ]:
!pip install OpenAI

In [ ]:
!pip install yake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 2.1 MB/s eta 0:00:00


In [ ]:
# prompt: I want to extract all the words that are in topics generated by topic_model_SBERT3, please generate code for this
topic_words = []
for i in range(len(topic_model_SBERT3.get_topic_info())):
  topic = topic_model_SBERT3.get_topic(i)
  if topic:
    topic_words.extend([word for word, prob in topic])
print(set(topic_words))

{'card', 'zs', 'experiences', 'gpu', 'bytedances', 'client', 'generative ai', 'group', 'employee', 'funds', 'netflix', 'digital', 'perception', 'tiktok', 'ai models', 'machine', 'develop', 'apple products', 'inspire creativity enrich', 'research', 'angeles county', 'c3', 'ymca', 'amd', 'products', 'accounting', 'data engineering', 'compiler', 'language', 'patients', 'enrich', 'customers', 'computer', 'new', 'sap', 'merchants', 'scientific', 'harvard', 'performance', 'dita', 'model', 'express', 'board', 'student', 'clinical', 'c3 ai', 'payments', 'staff', 'adobe', 'candidates', 'los angeles county', 'fair chance', 'ability', 'data science', 'development', 'racial', 'credit', 'aiml', 'denver', 'instruction', 'account', 'technical', 'hire', 'volunteer', 'ai', 'large', 'clients', 'youth', 'creativity enrich life', 'school', 'finance', 'generative', 'learning', 'field computational', 'physics', 'scientist', 'campus', 'engineering', 'machine learning', 'venmo', 'multimodal', 'investment', 'u

In [ ]:
# prompt: i want to download topic_words

import pandas as pd

topic_words = []
for i in range(len(topic_model_SBERT3.get_topic_info())):
  topic = topic_model_SBERT3.get_topic(i)
  if topic:
    topic_words.extend([word for word, prob in topic])

df_topic_words = pd.DataFrame(list(set(topic_words)), columns=['topic_words'])
df_topic_words.to_csv('topic_words.csv', index=False)
files.download('topic_words.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

LDA with Gensim

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Text Preprocessing Function
def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\W', ' ', text)  # Remove non-word characters
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Stopword removal
    return tokens

In [ ]:
# Preprocess all documents
texts = [preprocess_text(doc) for doc in docs]

# Create Dictionary and Corpus
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=5, no_above=0.8)  # Remove very rare and common words
corpus = [dictionary.doc2bow(text) for text in texts]

# Train LDA Model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=20,
                     random_state=42, passes=10, alpha='auto', eta='auto')

# Evaluate Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model_lda.get_coherence()

print(f'Coherence Score: {coherence_score}')
lda_model.print_topics(num_words=10)

Coherence Score: 0.38551667184941724


[(0,
  '0.016*"language" + 0.015*"ai" + 0.012*"data" + 0.011*"development" + 0.008*"solutions" + 0.008*"design" + 0.008*"software" + 0.007*"models" + 0.007*"skills" + 0.006*"learning"'),
 (1,
  '0.022*"learning" + 0.019*"models" + 0.018*"data" + 0.016*"machine" + 0.011*"ai" + 0.009*"science" + 0.007*"research" + 0.006*"strong" + 0.006*"model" + 0.006*"techniques"'),
 (2,
  '0.014*"research" + 0.012*"paypal" + 0.010*"ai" + 0.010*"merchants" + 0.010*"computer" + 0.009*"meta" + 0.008*"benefits" + 0.008*"consumers" + 0.007*"help" + 0.007*"global"'),
 (3,
  '0.089*"data" + 0.015*"business" + 0.013*"analysis" + 0.010*"analytics" + 0.008*"statistical" + 0.007*"ability" + 0.007*"tools" + 0.007*"using" + 0.007*"science" + 0.007*"skills"'),
 (4,
  '0.013*"research" + 0.011*"financial" + 0.008*"data" + 0.006*"position" + 0.006*"support" + 0.006*"skills" + 0.005*"including" + 0.005*"analysis" + 0.004*"relevant" + 0.004*"andor"'),
 (5,
  '0.001*"aws" + 0.001*"data" + 0.001*"including" + 0.000*"cust

In [ ]:
import openai
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Setting up a client for ChatGPT with API
client = openai.OpenAI(api_key="xx")
# Fine-tune topic embeddings with GPT
embedding_model = OpenAIBackend(client, "all-MiniLM-L6-v2")
embeddings = embedding_model.encode(docs, show_progress_bar=True)

In [ ]:
import openai
import numpy as np
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

client = openai.OpenAI(api_key="xx")

# Generate OpenAI embeddings
def get_openai_embeddings(texts, model="text-embedding-3-small"):
    response = openai.embeddings.create(input=texts, model=model)
    return np.array([embedding.embedding for embedding in response.data])

# Compute embeddings
embeddings = get_openai_embeddings(docs)

In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import openai
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

#Umap model to ensure reproducibility
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

#HDBSCAN_model to control the number of topics created
hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

#Count Vectorizer to remove stopwords and include bigrams and trigrams
stop_words = stopwords.words("english")  # Load extensive stopword list
vectorizer_model = CountVectorizer(stop_words=stop_words, ngram_range=(1, 3))

#Different representation models
# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-3.5
prompt = """
I have a topic that contains the following documents:
Job descriptions
The topic is described by the following keywords: competences, skills

Based on the information above, extract a short but highly descriptive topic label of at most 3 words. Make sure it is in the following format:
topic: <topic label>
"""
openai_model = OpenAI(client, model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "OpenAI": openai_model,
    "MMR": mmr_model,
    "POS": pos_model
}


In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Pipeline models
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(docs, embeddings)

# Show topics
topic_model.get_topic_info()

In [ ]:
# Pre-calculate embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(docs, show_progress_bar=True)

# UMAP model for reproducibility
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

# HDBSCAN model to control the number of topics
hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Count Vectorizer to remove stopwords and include bigrams and trigrams
stop_words = stopwords.words("english")  # Load stopword list
vectorizer_model = CountVectorizer(stop_words=stop_words, ngram_range=(1, 3))

# Representation models
keybert_model = KeyBERTInspired()
pos_model = PartOfSpeech("en_core_web_sm")
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-3.5 for topic labeling
prompt = """
I have a topic that contains the following documents:
Job descriptions
The topic is described by the following keywords: competences, skills

Based on the information above, extract a short but highly descriptive topic label of at most 3 words. Make sure it is in the following format:
topic: <topic label>
"""
openai_model = OpenAI(client, model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "OpenAI": openai_model,
    "MMR": mmr_model,
    "POS": pos_model
}

# Initialize BERTopic **without** embedding_model (since we precomputed embeddings)
topic_model = BERTopic(
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,
  top_n_words=10,
  verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(docs, embeddings)

# Show topics
topic_model.get_topic_info()
